# Tarea 3 - Optimización
### Método de Newton con Hessiano Modificado.
#### Por: Gustavo Hernández Angeles


### **Preparación**

#### Funciones de prueba y módulos

In [20]:
import numpy as np

# Definimos las funciones de prueba.
def Esfera(x):
    """
    f(x) = \\sum x_i^2
    """
    
    return np.sum(x**2)


def Rosenbrock(x):
    terminos = [
        100 * (x[i + 1] - x[i] ** 2) ** 2 + (1 - x[i]) ** 2
        for i in range(len(x) - 1)
    ]
    terminos = np.array(terminos)
    return np.sum(terminos)


def Beale(x):
    assert len(x) == 2, f"La función Beale es de dos variables, se introdujeron {len(x)}."
    (x1, x2) = x
    return (
        (1.5 - x1 + x1 * x2) ** 2
        + (2.25 - x1 + x1 * x2**2) ** 2
        + (2.625 - x1 + x1 * x2**3) ** 2
    )

#### Funciones para calcular Hessiano y Gradiente

In [21]:

def hess_f(x, f, h=1e-5):
    """
    Calcula la matriz Hessiana de f en el punto x usando diferencias finitas centrales.
    
    Input:
    f:  función escalar que recibe un vector x
    x:  punto donde se evalúa el Hessiano (numpy array)
    h:  paso pequeño para diferencias finitas

    Output:
    H:  matriz Hessiana (numpy array)
    """
    n = len(x)
    H = np.zeros((n, n))
    I = np.eye(n)  # Matriz identidad para cambiar un elemento a la vez

    for i in range(n):
        for j in range(n):
            x_ij = x + h * (I[i] + I[j])
            x_i = x + h * I[i]
            x_j = x + h * I[j]
            
            f_ij = f(x_ij)  # f(x + h_i + h_j)
            f_i = f(x_i)  # f(x + h_i)
            f_j = f(x_j)  # f(x + h_j)
            f_0 = f(x)  # f(x)

            H[i, j] = (f_ij - f_i - f_j + f_0) / (h**2)  # Segunda derivada cruzada

    return 0.5*(H + H.T)

def grad_f(x, f, h):
    """
    Input:
        f: Función.
        x: Punto a evaluar (numpy array).
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        grad: Valor del gradiente (numpy array).
    """
    
    # Inicializa el gradiente
    grad = np.zeros(len(x))

    # Itera sobre cada componente
    for i in range(len(x)):
        # Copia de x
        x_i = np.copy(x)

        # Se suma el espaciamiento solo en la i-esima componente
        x_i[i] = x_i[i] + h
        
        # Se calcula la i-esima componente del gradiente
        grad[i] = (f(x_i) - f(x)) / h
    return grad

#### Condicionar el hessiano

In [22]:
def condicionar_hessiano(hessiano, w = 1e-6, lam = 1e-6, max_iter = 10):
    """
    Asegura que el Hessiano sea definido positivo y bien condicionado
    Input:
        hessiano: Matriz Hessiana (numpy n x n)
        w: Cota menor de min_e/max_e.
        lambda: Minimum eigenvalue after adjustment (ensures positive definiteness).
    Output:
        Hessiano ajustado (numpy n x n)
    """
    
    n = hessiano.shape[0]
    H = np.copy(hessiano)
    
    # Asegura definida positiva
    eigenvals = np.linalg.eigvals(H)
    min_e = np.min(eigenvals)
    if min_e < lam:
        # Añade lo suficiente para que su minimo valor propio sea lam
        H += (lam - min_e) * np.eye(n)
        eigenvals = np.linalg.eigvals(H)  # Actualiza eigenvalores
    
    # Asegura la condición min_e / max_e >= w
    max_e = np.max(eigenvals)
    if max_e < 1e-12:  # Avoid division by near-zero
        max_e = 1e-12
    min_e = np.min(eigenvals)
    if min_e/ max_e < w:
        # min_e >= w * max_e
        objetivo_min_e = w * max_e
        H += (objetivo_min_e - min_e) * np.eye(n)
    
    
    return H

#### Método de Newton con Hessiano Modificado

In [23]:
def newton_hess_modificado(f, x, alfa, max_iter, epsilon, h):
    """
    Input:
        f: Función objetivo.
        x: Punto inicial x_0.
        alfa: Learning rate.
        max_iter: Número máximo de iteraciones.
        epsilon: Criterio de convergencia.
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        x like: Punto solución aproximada.
    """
    
    x_k = np.copy(x)
    convergencia = False
    
    for i in range(max_iter):
        # Obtenemos p_k
        hessiano = hess_f(x_k, f, h)
        hessiano = condicionar_hessiano(hessiano)
        grad = grad_f(x_k, f, h)
        p_k = -np.linalg.solve(hessiano,grad)
        
        # Actualizamos la solución
        x_k = x_k + alfa * p_k 
        
        # Evalua la convergencia
        convergencia = max(abs(p_k)) < epsilon
        if convergencia:
            print(f"La función {f.__name__} converge en la iteracion: {i}")
            break

    if not convergencia:
        print(f"No se cumplio la convergencia en {max_iter} iteraciones.")

    return x_k

### **Resultados**

#### Función Esfera

In [24]:
n = 4
x = np.array(n*[4], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-4
h = 10e-6

xsol = newton_hess_modificado(Esfera, x, alfa, max_iter, epsilon, h)

# Comparación con resultado real
solucion = np.array(n*[0], dtype=float)

print(f"Vector aprox: {xsol}")
print(f"Vector solución: {solucion}")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")

La función Esfera converge en la iteracion: 1
Vector aprox: [-5.e-06 -5.e-06 -5.e-06 -5.e-06]
Vector solución: [0. 0. 0. 0.]
Magnitud del vector error: 1.0000000000000453e-05


#### Función de Rosenbrock

In [25]:
n = 4
x = np.array(4*[2], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-4
h = 10e-6

xsol = newton_hess_modificado(Rosenbrock, x, alfa, max_iter, epsilon, h)

# Comparación con resultado real
solucion = np.array(n*[1], dtype=float)

print(f"Vector aprox: {xsol}")
print(f"Vector solución: {solucion}")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")

La función Rosenbrock converge en la iteracion: 10
Vector aprox: [0.99899826 0.99800253 0.99601405 0.992039  ]
Vector solución: [1. 1. 1. 1.]
Magnitud del vector error: 0.009179253304168914


#### Función de Beale

In [26]:
x = np.array([1,1], dtype=float)
alfa = 0.05
max_iter = 15_000
epsilon = 10e-4
h = 10e-6

xsol = newton_hess_modificado(Beale, x, alfa, max_iter, epsilon, h)

# Comparación con resultado real
solucion = np.array([3, 0.5], dtype=float)

print(f"Vector aprox: {xsol}")
print(f"Vector solución: {solucion}")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")

C:\Users\Gus\AppData\Local\Temp\ipykernel_26088\876322960.py:27: RuntimeWarning: overflow encountered in scalar power
  + (2.625 - x1 + x1 * x2**3) ** 2
C:\Users\Gus\AppData\Local\Temp\ipykernel_26088\3658465514.py:28: RuntimeWarning: invalid value encountered in scalar subtract
  H[i, j] = (f_ij - f_i - f_j + f_0) / (h**2)  # Segunda derivada cruzada


LinAlgError: Array must not contain infs or NaNs